# Classer ses iris (KNN)

David Scanu 

* v4 - Exporte notre modèle avec **pickle**


In [4]:
from sklearn import datasets
import numpy as np

# import some data to play with
iris = datasets.load_iris() # nd.array

In [5]:
# Features
X = iris.data

# Target
y = iris.target

In [6]:
# Feature Selection
from sklearn.feature_selection import SelectKBest, chi2

selector_kb = SelectKBest(chi2, k=2)
selector_kb.fit(X, y)
selector_kb.get_support() # Retourne une seule variable/colonne

array([False, False,  True,  True])

In [7]:
# Affiche les colonnes restantes
X_col_names_array = np.array(iris.feature_names)[selector_kb.get_support()]
X_col_names = X_col_names_array.tolist()
X_col_names

['petal length (cm)', 'petal width (cm)']

In [8]:
# Feature Selection
X_kb = selector_kb.transform(X)

In [9]:
# Split des données
from sklearn.model_selection import train_test_split

# Features
X_train, X_test, y_train, y_test = train_test_split(X_kb, y, test_size=0.2, random_state=3)

print(f"Nombre d'exemples d'entrainement X : {X_train.shape[0]}")
print(f"Nombre d'exemples de test X : {X_test.shape[0]}")
print(' ')
print(f"Nombre d'exemples d'entrainement Y : {y_train.shape[0]}")
print(f"Nombre d'exemples de test Y : {y_test.shape[0]}")

Nombre d'exemples d'entrainement X : 120
Nombre d'exemples de test X : 30
 
Nombre d'exemples d'entrainement Y : 120
Nombre d'exemples de test Y : 30


In [10]:
# Mise à l'échelle
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

In [11]:
# Modèle
from sklearn.neighbors import KNeighborsClassifier

In [12]:
# Grid Search CV pour fabriquer notre modèle avec les meilleurs paramètres
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors' : np.arange(1, 20), 'metric' : ['euclidean', 'manhattan']}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train_scaler, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [13]:
# Meilleur score (sur la validation croisée)
grid.best_score_

0.9583333333333334

In [14]:
# Meilleurs paramètres
grid.best_params_

{'metric': 'euclidean', 'n_neighbors': 10}

In [15]:
# Meilleur modèle
model_best = grid.best_estimator_

In [16]:
# Score du meilleur modèle (sur les données de test)
model_best.score(X_test_scaler, y_test)

0.9666666666666667

## Sauvegarde

In [17]:
import pickle

dict_export = {}
dict_export['model'] = model_best
dict_export['X_col_names'] = X_col_names
dict_export['y_names'] = iris.target_names
dict_export['scaler'] = scaler

pickle_out = open("app/model.pkl","wb")
pickle.dump(dict_export, pickle_out)
pickle_out.close()
